In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import networkx as nx


In [2]:
df = pd.read_csv('exported_data/normalized_13.df')
# df = pd.read_csv('exported_data/normalized_3.df')


PREFIX = "mteb_ds"

In [3]:
datasets = []

for idx, val in df['datasets'].apply(lambda x: sorted(eval(x))).items():
    datasets.extend(val)
    
datasets = list(set(datasets))
print(datasets)

['mteb/stsbenchmark-sts/test/embeddings.npy', 'mteb/sts13-sts/test/embeddings.npy', 'mteb/banking77/test/embeddings.npy', 'snli/validation/embeddings.npy', 'mteb/biosses-sts/test/embeddings.npy', 'snli/test/embeddings.npy', 'mteb/amazon_polarity/test/embeddings.npy', 'mteb/sts12-sts/test/embeddings.npy', 'mteb/stsbenchmark-sts/validation/embeddings.npy', 'mteb/sts15-sts/test/embeddings.npy', 'mteb/sickr-sts/test/embeddings.npy', 'mteb/sts14-sts/test/embeddings.npy']


In [4]:
df.head()

,Unnamed: 0,id,date,model_1,model_2,d_1,d_2,dataset_filter,datasets,I(X_1->X_2),...,optimize_mu,cond_modes,marg_modes,use_tanh,init_std,ff_residual_connection,ff_activation,ff_layer_norm,ff_layers,ff_dim_hidden
0,0,dc785306-8cae-4db1-9d55-a8f6434f3803,2024-04-07 16:25:17,../output/sentence-transformers/average_word_e...,../output/sentence-transformers/msmarco-bert-c...,300,768,NaN,"{'mteb/sts13-sts/test/embeddings.npy', 'mteb/s...",292.661631,...,True,8,8,True,0.01,False,relu,True,2,0
1,0,5320b1f9-20db-4cbb-b774-c79f9ab1ad90,2024-04-07 14:39:08,../output/llmrails/ember-v1,../output/intfloat/e5-small,1024,384,NaN,"{'snli/validation/embeddings.npy', 'mteb/sts14...",178.770718,...,True,8,8,True,0.01,False,relu,True,2,0
2,0,f370bf89-e69b-494e-b157-162c76bdd4cf,2024-04-07 19:22:12,../output/sentence-transformers/msmarco-bert-c...,../output/google/gemma-7b-it,768,3072,NaN,"{'mteb/sts14-sts/test/embeddings.npy', 'mteb/s...",1800.378266,...,True,8,8,True,0.01,False,relu,True,2,0
3,0,a52b3637-e9eb-4fee-810e-054aaa424069,2024-04-07 17:06:28,../output/google/gemma-7b,../output/HuggingFaceM4/tiny-random-LlamaForCa...,3072,16,NaN,"{'mteb/stsbenchmark-sts/test/embeddings.npy', ...",5.066857,...,True,8,8,True,0.01,False,relu,True,2,0
4,0,2b4d8f82-52bf-454e-a6b4-f94e4fd25af1,2024-04-07 14:25:10,../output/sentence-transformers/sentence-t5-xl,../output/sentence-transformers/LaBSE,768,768,NaN,"{'mteb/banking77/test/embeddings.npy', 'mteb/s...",396.074551,...,True,8,8,True,0.01,False,relu,True,2,0


In [5]:
df['I(X_1->X_2)/H(X_2)'] = df['I(X_1->X_2)'] / df['H(X_2)']
df['I(X_1->X_2)/d_1'] = df['I(X_1->X_2)'] / df['d_1']
df['I(X_1->X_2)/d_2'] = df['I(X_1->X_2)'] / df['d_2']

df['model_1'] = df['model_1'].apply(lambda x: "/".join(x.split('/')[-2:]))
df['model_2'] = df['model_2'].apply(lambda x: "/".join(x.split('/')[-2:]))




In [6]:
df = df.groupby(['model_1', 'model_2']).first().reset_index()

In [7]:

df = df[~(df['model_1'] == "jinaai/jina-embedding-s-en-v1")]
df = df[~(df['model_2'] == "jinaai/jina-embedding-s-en-v1")]

In [8]:
sns.set_style("whitegrid")
METRIC = 'I(X_1->X_2)/d_2'
# METRIC = 'I(X_1->X_2)/H(X_2)'
# METRIC = 'I(X_1->X_2)'
from matplotlib import patheffects

cmap =sns.color_palette("coolwarm", as_cmap=True)


table = df
table = table[["model_1", "model_2", METRIC]].pivot("model_1", "model_2",METRIC).fillna(0)

# count number of 0 per row
n_0 = table.apply(lambda x: x[x == 0].count(), axis=1)

# count number of 0 per column
n_0_col = table.apply(lambda x: x[x == 0].count(), axis=0)

# remove rows with more than 50% of 0
# table = table[n_0 < table.shape[1] // 2]

# remove columns with more than 50% of 0
# table = table.loc[:, n_0_col < table.shape[0] // 2]

# make square
# table = table.loc[table.index.intersection(table.columns)]



# remove lines with only 0

/tmp/ipykernel_44065/3807154875.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  table = table[["model_1", "model_2", METRIC]].pivot("model_1", "model_2",METRIC).fillna(0)


In [9]:
table

model_2,BAAI/bge-base-en-v1.5,GritLM/GritLM-7B,HuggingFaceM4/tiny-random-LlamaForCausalLM,NousResearch/Llama-2-7b-hf,Salesforce/SFR-Embedding-Mistral,SmartComponents/bge-micro-v2,TaylorAI/gte-tiny,WhereIsAI/UAE-Large-V1,avsolatorio/GIST-Embedding-v0,croissantllm/CroissantCool,...,sentence-transformers/average_word_embeddings_glove.6B.300d,sentence-transformers/average_word_embeddings_komninos,sentence-transformers/gtr-t5-base,sentence-transformers/gtr-t5-large,sentence-transformers/gtr-t5-xl,sentence-transformers/msmarco-bert-co-condensor,sentence-transformers/sentence-t5-large,sentence-transformers/sentence-t5-xl,thenlper/gte-base,thenlper/gte-large
model_1,,,,,,,,,,,,,,,,,,,,,
BAAI/bge-base-en-v1.5,1.408755,0.556619,0.216323,0.658130,0.627063,0.505610,0.518470,0.672778,1.279503,0.580892,...,0.498936,0.445808,0.476813,0.465046,0.446726,0.527686,0.550271,0.515953,1.076783,0.629245
GritLM/GritLM-7B,0.562268,0.928033,0.370918,0.866750,0.665229,0.512743,0.502282,0.613329,0.591918,0.780834,...,0.556971,0.537035,0.506658,0.514760,0.513704,0.508990,0.594949,0.564597,0.519053,0.572204
HuggingFaceM4/tiny-random-LlamaForCausalLM,0.073016,0.063781,1.128528,0.089295,0.033481,0.071567,0.069533,0.067744,0.069579,0.089979,...,0.115742,0.105286,0.056679,0.058115,0.056857,0.060608,0.065630,0.060249,0.061303,0.061814
NousResearch/Llama-2-7b-hf,0.523777,0.692050,0.364495,1.022687,0.548256,0.475209,0.462156,0.576038,0.546495,0.759894,...,0.533015,0.499705,0.464317,0.459331,0.471243,0.472029,0.552098,0.534061,0.488254,0.522408
Salesforce/SFR-Embedding-Mistral,0.583908,0.644836,0.298796,0.729444,1.141443,0.541588,0.532534,0.648821,0.612515,0.638281,...,0.540469,0.499941,0.548169,0.555192,0.573165,0.526496,0.640687,0.632684,0.521461,0.594409
SmartComponents/bge-micro-v2,0.511468,0.547267,0.206421,0.635582,0.620145,1.410436,0.798591,0.553717,0.549635,0.571141,...,0.520413,0.468280,0.443943,0.413541,0.432264,0.480542,0.492247,0.442565,0.484923,0.507655
TaylorAI/gte-tiny,0.547937,0.548944,0.198077,0.640014,0.607331,0.806125,1.424008,0.563739,0.574487,0.578049,...,0.503811,0.460555,0.442818,0.413936,0.422918,0.461597,0.491881,0.461073,0.510063,0.535302
WhereIsAI/UAE-Large-V1,0.625907,0.556661,0.199992,0.634317,0.646123,0.501665,0.507954,1.413947,0.667389,0.588979,...,0.488767,0.440581,0.465090,0.460625,0.457000,0.494160,0.573612,0.515566,0.567829,1.131927
avsolatorio/GIST-Embedding-v0,1.252645,0.559947,0.207721,0.655228,0.630045,0.524688,0.532512,0.680945,1.427954,0.588988,...,0.494188,0.449655,0.459249,0.451516,0.458153,0.513908,0.561671,0.514599,1.154581,0.618793


In [10]:
def sanitize_metric_name(metric):
    return metric.replace('/', '_').replace(' ', '_').replace('(', '').replace(')', '').replace('->', 'to').replace('(', '').replace(')', '')


In [11]:
from scipy.cluster.hierarchy import linkage
link = linkage(table, method="ward")

cluster = sns.clustermap(table, row_linkage=link, col_linkage=link, figsize=(20, 20), cmap='flare', annot=True, fmt=".1f", vmax=1)

# make labels bigger and bold and rename in model X and model Y
cluster.ax_heatmap.set_xticklabels(cluster.ax_heatmap.get_xticklabels(), fontsize=15, fontweight='bold')
cluster.ax_heatmap.set_yticklabels(cluster.ax_heatmap.get_yticklabels(), fontsize=15, fontweight='bold')
cluster.ax_heatmap.set_xlabel("Model Y", fontsize=15, fontweight='bold')
cluster.ax_heatmap.set_ylabel("Model X", fontsize=15, fontweight='bold')

path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_mis_graph_clustermap_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
cluster.savefig(path, bbox_inches='tight', dpi=300)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fc95913e0d0>>
Traceback (most recent call last):
  File "/usr/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fc95913e0d0>>
Traceback (most recent call last):
  File "/usr/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fc95913e0d0>>
Traceback (most recent call last):
  File "/usr/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_fra

Error in callback <function _draw_all_if_interactive at 0x7fc9230d3100> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



Error in callback <function flush_figures at 0x7fc91f431260> (for post_execute), with arguments args (),kwargs {}:


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fc95913e0d0>>
Traceback (most recent call last):
  File "/usr/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
# Community detection

from networkx.algorithms.community import greedy_modularity_communities

G = nx.from_pandas_adjacency(table, create_using=nx.DiGraph)
communities = list(greedy_modularity_communities(G))


    
    


In [ ]:
from netgraph import Graph, InteractiveGraph

from networkx.algorithms.community import girvan_newman, modularity_max, louvain_communities

G= nx.from_pandas_adjacency(table, create_using=nx.DiGraph)
G.remove_edges_from(nx.selfloop_edges(G))

communities = louvain_communities(G, resolution=1.1)
communities = list(communities)
# get a discrete color map

cmap = sns.color_palette("flare", as_cmap=True)

avg_weight = {n : np.median([d[2]['weight'] for d in G.out_edges(n, data=True)]) for n in G.nodes()}
avg_income = {n : np.median([d[2]['weight'] for d in G.in_edges(n, data=True)]) for n in G.nodes()}

# normalize the average weight

node_to_community = {node: i for i, community in enumerate(communities) for node in community}

# node color using a color map
# node_color = {node: cmap(i) for i, community in enumerate(communities) for node in community}

# make average out going weight the node color
node_color = {node: cmap(avg_weight[node]) for node in G.nodes()}
node_edge_color = {node: cmap(avg_income[node]) for node in G.nodes()}


node_labels = {node: node for node in G.nodes()}

edge_color = {edge: G.edges[edge]['weight'] for edge in G.edges()}
# normalize edge 
edge_color = {edge: cmap(edge_color[edge]) for edge in edge_color}
max_edge = max(e[2]['weight'] for e in G.edges(data=True))


# normalize edge alpha
min_alpha = 0.01
max_alpha = 0.4
edge_alpha = {edge: G.edges[edge]['weight'] for edge in G.edges()}
edge_alpha = {edge: (edge_alpha[edge] - min(edge_alpha.values())) / (max(edge_alpha.values()) - min(edge_alpha.values())) * (max_alpha - min_alpha) + min_alpha for edge in edge_alpha}

# edge width
min_edge_width = 0.01
max_edge_width = 0.7
edge_width = {edge: G.edges[edge]['weight'] for edge in G.edges()}
edge_width = {edge: (edge_width[edge] - min(edge_width.values())) / (max(edge_width.values()) - min(edge_width.values())) * (max_edge_width - min_edge_width) + min_edge_width for edge in edge_width}


fig, ax = plt.subplots(figsize=(30, 30))

graph = Graph(G, node_layout_kwargs=dict(node_to_community=node_to_community), node_layout="community", node_color=node_color, node_labels=node_labels, edge_color=edge_color, ax=ax, node_label_fontdict={'fontsize': 15, 'fontweight': 'bold'}, node_edge_color=node_edge_color, edge_layout="curved", edge_alpha=edge_alpha, arrows=True)



# add white contour to all texts in the figure
for text in plt.gca().texts:
    text.set_path_effects([patheffects.Stroke(linewidth=4, foreground='white'), patheffects.Normal()])
    
# add color bar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=max_edge))
sm._A = []
# cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', shrink=0.5, pad=0.05)

path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_mis_graph_community_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
# fig.tight_layout()
# fig.savefig(path, bbox_inches='tight', dpi=300)

print("DOESNT SAVE ANYTHING SEE MODEL CLUSTERING!!!")



In [ ]:
predictive_power = table.mean(axis=1).sort_values(ascending=False).to_frame()
predictive_power['community'] = predictive_power.index.map(node_to_community)

# keep top 1 for each community
top_1 = predictive_power.groupby('community').head(1).sort_values(by=0, ascending=False).head(5)

# get pair with the lowest predictive power

top_table = table.loc[top_1.index, top_1.index]


unrelated_pairs = []
for k, m1 in enumerate(top_table.index):
    for j, m2 in enumerate(top_table.columns):
        if k < j:
            v = max(top_table.loc[m1, m2], top_table.loc[m2, m1])
            unrelated_pairs.append((m1, m2, v, predictive_power.loc[m1, 0], predictive_power.loc[m2, 0]))
            
unrelated_pairs = sorted(unrelated_pairs, key=lambda x: x[2], reverse=True)
unrelated_pairs
    
    
text = ""
for pair in unrelated_pairs:
    text += f"'{pair[0]} {pair[1]}' "
    
print(text)





In [ ]:

# classifcation_df = pd.read_csv('exported_data/classification_2.csv')
classifcation_df = pd.read_csv('exported_data/classification_many.csv')

display(classifcation_df)

classifcation_df['model'] = classifcation_df['model'].apply(lambda x: eval(x)[0])

In [ ]:
classification_ranking = classifcation_df.groupby('model')['success'].mean().sort_values(ascending=False).to_frame()

classification_ranking['ranking'] = "classification"
classification_ranking['rank'] = classification_ranking['success'].rank(ascending=False)
# rename success to value
classification_ranking = classification_ranking.rename(columns={'success': 'value'})
classification_ranking = classification_ranking.reset_index()



informativeness_ranking = table.median(axis=1).sort_values(ascending=False).to_frame()

# rename 0 to informativeness
informativeness_ranking = informativeness_ranking.rename(columns={0: 'value'})

informativeness_ranking['ranking'] = "informativeness"
informativeness_ranking['rank'] = informativeness_ranking['value'].rank(ascending=False)
informativeness_ranking = informativeness_ranking.reset_index()

informativeness_ranking = informativeness_ranking.rename(columns={'index': 'model', 'model_1': 'model'})


G= nx.from_pandas_adjacency(table, create_using=nx.DiGraph)
G.remove_edges_from(nx.selfloop_edges(G))

communities = louvain_communities(G, resolution=1.1)
communities = list(communities)

ranking = pd.concat([classification_ranking, informativeness_ranking], axis=0)

# model to community
node_to_community = {node: i for i, community in enumerate(communities) for node in community}

ranking['community'] = ranking['model'].apply(lambda x: node_to_community[x] if x in node_to_community else -1)

display(informativeness_ranking)

In [ ]:

toplot_info = ranking[ranking['ranking'] ==  "informativeness"]
toplot_classif = ranking[ranking['ranking'] ==  "classification"]


# rank both according to toplot_info value
toplot_info = toplot_info.sort_values(by="value", ascending=False)
info_models = toplot_info['model'].values
classifs_models = toplot_classif['model'].values


# make intersection
info_models = set(info_models)
classifs_models = set(classifs_models)
models = list(info_models.intersection(classifs_models))


print(info_models)
print(classifs_models)

In [ ]:



# sort according to informativeness
models = sorted(models, key=lambda x: toplot_info.loc[toplot_info['model'] == x, 'value'].values[0], reverse=True)
print(models)
toplot_classif = toplot_classif.set_index('model').loc[models].reset_index()
toplot_info = toplot_info[toplot_info['model'].isin(models)]

spectral_cmap = sns.color_palette("Spectral", as_cmap=True)
model_colors = {m : k for k, m in enumerate(models)}
# normalize colors
model_colors = {m: spectral_cmap(k / len(model_colors)) for m, k in model_colors.items()}

toplot_info['color'] = toplot_info['model'].map(model_colors)
toplot_classif['color'] = toplot_classif['model'].map(model_colors)


# make ranked bar plot
fig, ax = plt.subplot_mosaic([['ranking_classif', 'ranking_informativeness']], sharey=True, sharex=False, figsize=(15, 7))

# horizontal bar plot, with shared y axis
# share y axis

hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']*2

sns.barplot(data=toplot_info, y="model", x="value", ax=ax['ranking_informativeness'], orient='h', palette=model_colors, hue="model")
for i, bar in enumerate(ax['ranking_informativeness'].patches):
    # hatche based on community
    bar.set_hatch(hatches[toplot_info.iloc[i]['community']])

# set xlim
ax['ranking_informativeness'].set_xlim(0.3, 0.65)

sns.barplot(data=toplot_classif, y="model", x="value", ax=ax['ranking_classif'], orient='h', hue="model", palette=model_colors)

for i, bar in enumerate(ax['ranking_classif'].patches):
    # hatche based on community
    bar.set_hatch(hatches[toplot_classif.iloc[i]['community']])

# set xlim
ax['ranking_classif'].set_xlim(0.5, 0.7)

# share y axis:
ax['ranking_informativeness'].sharey(ax['ranking_classif'])
ax['ranking_informativeness'].tick_params(labelleft=False)
ax['ranking_informativeness'].set_ylabel("")

# make yticks bigger
ax['ranking_classif'].tick_params(axis='y', labelsize=15)

# remove xticks
ax['ranking_classif'].tick_params(labelbottom=False)
ax['ranking_classif'].set_xlabel("")
ax['ranking_informativeness'].tick_params(labelbottom=False)
ax['ranking_informativeness'].set_xlabel("")

# remvoe ylabels
ax['ranking_classif'].set_ylabel("")

# set titles
ax['ranking_classif'].set_title("Classification ranking", fontsize=15, fontweight='bold')
ax['ranking_informativeness'].set_title("Informativeness ranking", fontsize=15, fontweight='bold')




fig.tight_layout()
path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/nlp_models_rankings_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(path, bbox_inches='tight', dpi=300)

######## 


# deduplicate the ranking
for_pivot = ranking.groupby(['model', 'community', 'ranking']).first().reset_index()



ranking_pivot = for_pivot.pivot(columns="ranking", values="rank", index=['model', 'community']).reset_index()
# select models

display(ranking_pivot)

print(models)
ranking_pivot = ranking_pivot[ranking_pivot['model'].isin(models)]


# remove community -1
ranking_pivot = ranking_pivot[ranking_pivot['community'] != -1]
ranking_pivot['community'] = ranking_pivot['community'].astype(str)

# sort
ranking_pivot = ranking_pivot.sort_values(by="community")

# rename model and community
ranking_pivot = ranking_pivot.rename(columns={'model': 'Model', 'community': 'Community', 'classification': 'Classification', 'informativeness': 'Informativeness'})

fig, ax = plt.subplots(figsize=(15, 5))



g = sns.scatterplot(data=ranking_pivot, x="Classification", y="Informativeness", style="Community", hue="Model", s=600, ax=ax, legend=False, palette=model_colors)
# plot linear regression
sns.regplot(data=ranking_pivot, x="Classification", y="Informativeness", scatter=False, ax=ax)
# annotate with correlation
correlation = ranking_pivot['Classification'].corr(ranking_pivot['Informativeness'])
ax.annotate(f"Spearman correlation: {correlation:.2f}", xy=(0.5, 0.1), xycoords='axes fraction', fontsize=15, fontweight='bold')

# make labels bigger and bold and rename in model X and model Y
ax.set_xlabel("Classification ranking", fontsize=15, fontweight='bold')
ax.set_ylabel("Informativeness ranking", fontsize=15, fontweight='bold')


fig.tight_layout()
path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_nlp_models_ranking_correlation_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(path, bbox_inches='tight', dpi=300)








In [ ]:
# pivot the table
ranking_pivot = ranking.pivot(columns="ranking", values="rank", index=['model', 'community']).reset_index()
# remove community -1
ranking_pivot = ranking_pivot[ranking_pivot['community'] != -1]

ranking_pivot['community'] = ranking_pivot['community'].astype(str)


# sort
ranking_pivot = ranking_pivot.sort_values(by="community")

# rename model and community
ranking_pivot = ranking_pivot.rename(columns={'model': 'Model', 'community': 'Community', 'classification': 'Classification', 'informativeness': 'Informativeness'})

fig, ax = plt.subplots(figsize=(15, 5))
g = sns.scatterplot(data=ranking_pivot, x="Classification", y="Informativeness", style="Community", hue="Model", s=600, ax=ax)

# plot linear regression
sns.regplot(data=ranking_pivot, x="Classification", y="Informativeness", scatter=False, ax=ax)
# annotate with correlation
correlation = ranking_pivot['Classification'].corr(ranking_pivot['Informativeness'])
ax.annotate(f"Spearman correlation: {correlation:.2f}", xy=(0.5, 0.1), xycoords='axes fraction', fontsize=15, fontweight='bold')



# legend outside with 2 columns, below the plot
plt.legend(bbox_to_anchor=(.0, -0.15), loc=2, borderaxespad=0., ncol=3)

# rename x and y labels and make bold and bigger

ax.set_xlabel("Classification ranking", fontsize=15, fontweight='bold')
ax.set_ylabel("Informativeness ranking", fontsize=15, fontweight='bold')


# save the figure
path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_nlp_models_ranking_correlation_{sanitize_metric_name(METRIC)}_legend.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(path, bbox_inches='tight', dpi=300)




In [ ]:

# pivot the table

fig, ax = plt.subplots(figsize=(15, 5))

ranking_pivot = ranking.pivot(columns="ranking", values="value", index=['model', 'community']).reset_index()

sns.scatterplot(data=ranking_pivot, x="classification", y="informativeness", style="community", hue="model", s=200, ax=ax)

# plot linear regression
sns.regplot(data=ranking_pivot, x="classification", y="informativeness", scatter=False, ax=ax)
# annotate with correlation
correlation = ranking_pivot['classification'].corr(ranking_pivot['informativeness'])
ax.annotate(f"Correlation: {correlation:.2f}", xy=(0.5, 0.1), xycoords='axes fraction', fontsize=15, fontweight='bold')

# legend outside

plt.legend(bbox_to_anchor=(.0, -0.15), loc=2, borderaxespad=0., ncol=3)
path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_nlp_models_acc_info_correlation_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(path, bbox_inches='tight', dpi=300)




In [ ]:
toplot = ranking[ranking['ranking'] ==  "classification"]
toplot['community'] = toplot['community'].astype(str)
# remove community -1
toplot = toplot[toplot['community'] != "-1"]
fig, ax = plt.subplots(figsize=(10, 5))
toplot = toplot.sort_values(by="community")
sns.barplot(data=toplot, x="model", y="value", ax=ax, hue="community")
# rotate x labels
plt.xticks(rotation=90)
plt.ylim(0.5, 0.7)

# rename x and y labels and make bold and bigger
plt.xlabel("Model", fontsize=15, fontweight='bold')
plt.ylabel("Avg. Accuracy", fontsize=15, fontweight='bold')

path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_classifs_perfs_per_community_{sanitize_metric_name(METRIC)}.png")

In [ ]:
## Classification clustering

In [ ]:
full_classification = pd.read_csv('exported_data/classification_many.csv')
full_classification = full_classification[~full_classification['dataset'].str.contains('clinc') ]


full_classification['model'] = full_classification['model'].apply(lambda x: eval(x)[0])


In [ ]:

# add community to the classification
full_classification['community'] = full_classification['model'].apply(lambda x: str(node_to_community[x]) if x in node_to_community else -1)

# remove if community is -1
full_classification = full_classification[full_classification['community'] != -1]

In [ ]:
print(node_to_community)

In [ ]:
full_classification

In [ ]:

classification_perfs = full_classification.join(ranking.set_index('model'), on='model', how='left')



    
    
    

In [ ]:


# load mteb data

mteb = pd.read_csv('exported_data/df_mteb_avg.csv')
mteb

In [ ]:
mteb

import re

def extract_url_from_html_link(html):
    return re.findall(r'href=[\'"]?([^\'" >]+)', html)[0]


mteb['model'] = mteb['Model'].apply(extract_url_from_html_link).apply(lambda x: "/".join(x.split('/')[-2:]))


In [ ]:

avg_results = table.mean(axis=1).sort_values(ascending=False).to_frame().reset_index().rename(columns={0: METRIC, 'model_1': 'model'}).merge(mteb, on='model', how='left').sort_values(METRIC, ascending=False)

# remove unnamed columns
avg_results = avg_results.loc[:, ~avg_results.columns.str.contains('^Unnamed')]



In [ ]:

print(list(avg_results.columns))

METRICS = ['Average (56 datasets)', 'Classification Average (12 datasets)', 'Clustering Average (11 datasets)', 'Reranking Average (4 datasets)', 'Retrieval Average (15 datasets)', 'STS Average (10 datasets)'] + [METRIC]

METADATA = ['model', 'Rank', 'Model', 'Model Size (GB)', 'Embedding Dimensions', 'Max Tokens']

In [ ]:

sns.heatmap(avg_results[METRICS].corr(), annot=True, cmap='coolwarm')

In [ ]:
# melt
avg_results_pivot = avg_results.melt(id_vars=METADATA+[METRIC], value_vars=set(METRICS)-set(METRIC), var_name="Metric", value_name="Accuracy")
# remove metric about summarization


avg_results_pivot["Community"] = avg_results_pivot['model'].map(node_to_community)

# sns.relplot(data=avg_results_pivot, x=METRIC, y="Value", col="Metric", kind="scatter", col_wrap=3, height=10, aspect=1.5, hue="model", facet_kws={'sharey': False, 'sharex': False}, s=500)

g = sns.FacetGrid(data=avg_results_pivot, col="Metric", col_wrap=3, height=5, aspect=1.5, sharey=False, sharex=False)

def scatterplot_relplot(x, y, **kwargs):
    data = kwargs.pop('data')
    sns.scatterplot(x=x, y=y, data=data, ax=plt.gca(), **kwargs)
    # regplot
    sns.regplot(x=x, y=y, data=data, scatter=False, ax=plt.gca())
    
    # annotate with correlation
    correlation = data[x].corr(data[y])
    ax = plt.gca()
    ax.annotate(f"Correlation: {correlation:.2f}", xy=(0.5, 0.1), xycoords='axes fraction', fontsize=15, fontweight='bold')
    
g.map_dataframe(scatterplot_relplot, x=METRIC, y="Accuracy", hue="model", s=500, style="Community")

# change title template 
g.set_titles(template="{col_name}", size=15, fontweight='bold')

# make x and y labels bigger and bold
g.set_xlabels(METRIC, fontsize=15, fontweight='bold')
g.set_ylabels("Accuracy", fontsize=15, fontweight='bold')

path = Path(f"../../../papers/emir-embedding-comparison/fig/nlp/{PREFIX}_mteb_correlation_{sanitize_metric_name(METRIC)}.png")
# make sure the folder exists
path.parent.mkdir(parents=True, exist_ok=True)
g.savefig(path, bbox_inches='tight', dpi=300)

In [ ]:

# Correlation between metric and embedding dimensions
sns.scatterplot(data=avg_results, x=METRIC, y="Embedding Dimensions", hue="model", s=500, legend=False)


In [ ]:
# Make metadata table


metadata_table = avg_results.copy()[METADATA+METRICS]

metadata_table = metadata_table.drop(["Model", "Rank"], axis=1).rename(columns={'model': 'Model'})
metadata_table = metadata_table.set_index('Model')
metadata_table['Embedding Dimensions'] = df.groupby('model_2')['d_2'].first()

# rename I(X_1->X_2)/d_2 to Informativeness

metadata_table = metadata_table.rename(columns={METRIC: "Info."})
metadata_table = metadata_table.rename(columns={'Average (56 datasets)': 'Average', 'Classification Average (12 datasets)': 'Classification', 'Clustering Average (11 datasets)': 'Clustering', 'Reranking Average (4 datasets)': 'Reranking', 'Retrieval Average (15 datasets)': 'Retrieval', 'STS Average (10 datasets)': 'STS', "Embedding Dimensions": "Dim.", "Model Size (GB)" : "Size (GB)", "Max Tokens": "Max Tokens"})

# make \\url with hugginface url and model name
metadata_table = metadata_table.reset_index()
metadata_table['Model'] = metadata_table['Model'].apply(lambda x: x.replace('_', r'\_')).apply(lambda x: "\\href{https://huggingface.co/" + x + "}" + "{" + x + "}")
metadata_table = metadata_table.set_index('Model')

style = metadata_table.style.format({'Model Size (GB)': "{:.2f}", 'Embedding Dimensions': "{:.0f}",  "Max Tokens": "{:.0f}", "Info." : "{:.2f}", 'Average': "{:.2f}", 'Classification': "{:.2f}", 'Clustering': "{:.2f}", 'Reranking': "{:.2f}", 'Retrieval': "{:.2f}", 'STS': "{:.2f}", 'Dim.': "{:.0f}", 'Size (GB)': "{:.2f}", 'Max Tokens': "{:.0f}"}, na_rep="N/A")

# make Info. bold
style = style.set_properties(subset=["Info."], **{"bfseries" :""})

# escape
style = style.format_index()

path = Path(f"../../../papers/emir-embedding-comparison/tables/nlp/{PREFIX}_mteb_metadata_table_{sanitize_metric_name(METRIC)}.tex")
path.parent.mkdir(parents=True, exist_ok=True)
latex = style.to_latex(clines="skip-last;data", sparse_index=True, caption="Summary of the evaluated embedders with their performance on the MTEB benchmark along with their informativeness.", label="tab:nlp_metadata_table", hrules=True)

# add resizebox to the latex
latex = latex.replace("\\begin{tabular}", "\\resizebox{\\textwidth}{!}{ \\begin{tabular}")
latex = latex.replace("\\end{tabular}", "\\end{tabular}\n}")

# save the latex
path.write_text(latex)


In [ ]:

# ['llmrails/ember-v1', 'jamesgpt1/sf_model_e5', 'infgrad/stella-base-en-v2', 'WhereIsAI/UAE-Large-V1', 'avsolatorio/GIST-Embedding-v0', 'BAAI/bge-base-en-v1.5', 'thenlper/gte-base', 'thenlper/gte-large', 'intfloat/e5-large-v2', 'sentence-transformers/gtr-t5-large', 'sentence-transformers/gtr-t5-xl', 'sentence-transformers/sentence-t5-large', 'sentence-transformers/gtr-t5-base', 'sentence-transformers/sentence-t5-xl', 'TaylorAI/gte-tiny', 'SmartComponents/bge-micro-v2', 'intfloat/e5-small', 'sentence-transformers/msmarco-bert-co-condensor', 'intfloat/multilingual-e5-small', 'sentence-transformers/all-distilroberta-v1', 'princeton-nlp/sup-simcse-bert-base-uncased', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-mpnet-base-v2', 'izhx/udever-bloom-560m', 'sentence-transformers/LaBSE', 'sentence-transformers/average_word_embeddings_komninos', 'sentence-transformers/average_word_embeddings_glove.6B.300d', 'sentence-transformers/allenai-specter']

# dict model name to citation name

model_citation = {
    'llmrails/ember-v1': 'EMBER',
    'jamesgpt1/sf_model_e5': 'SF-Model',
    'infgrad/stella-base-en-v2': 'STELLA',
    'WhereIsAI/UAE-Large-V1': 'UAE-Large',
    'avsolatorio/GIST-Embedding-v0': 'GIST',
    'BAAI/bge-base-en-v1.5': 'BGE-Base',
    'thenlper/gte-base': 'GTE-Base',
    'thenlper/gte-large': 'GTE-Large',
    'intfloat/e5-large-v2': 'E5-Large',
    'sentence-transformers/gtr-t5-large': 'GTR-T5-Large',
    'sentence-transformers/gtr-t5-xl': 'GTR-T5-XL',
    'sentence-transformers/sentence-t5-large': 'Sentence-T5-Large',
    'sentence-transformers/gtr-t5-base': 'GTR-T5-Base',
    'sentence-transformers/sentence-t5-xl': 'Sentence-T5-XL',
    'TaylorAI/gte-tiny': 'GTE-Tiny',
    'SmartComponents/bge-micro-v2': 'BGE-Micro',
    'intfloat/e5-small': 'E5-Small',
    'sentence-transformers/msmarco-bert-co-condensor': 'MSMARCO-BERT',
    'intfloat/multilingual-e5-small': 'Multilingual-E5-Small',
    'sentence-transformers/all-distilroberta-v1': 'DistilRoBERTa',
    'princeton-nlp/sup-simcse-bert-base-uncased': 'SimCSE',
    'sentence-transformers/all-MiniLM-L6-v2': 'MiniLM-L6',
    'sentence-transformers/all-mpnet-base-v2': 'MPNet-Base',
    'izhx/udever-bloom-560m': 'UDEVER-Bloom',
    'sentence-transformers/LaBSE': 'LaBSE',
    'sentence-transformers/average_word_embeddings_komninos': 'Komninos',
    'sentence-transformers/average_word_embeddings_glove.6B.300d': 'GloVe',
    'sentence-transformers/all

